# 字符与编码
## 字符标识
unicode是目前最流行的字符标识标准，在该标准中以4~6个十六进制数字标识一个字符，例如A的unicode标识为U+0041（共2字节，第一个00表示空位，实际上仅需要一个字节表示即可，python3.3+会对str的内存表示做动态优化以达到最佳内存使用）
## 编码
编码表示在字符标识和字节序列之间转换时使用的算法，如UTF-8编码，A（U+0041）被编码为\x41\x00两个字节。

In [7]:
s = 'café'
print(len(s))
# 'é'被编码为\xc3\xa9两个字节,被存储为bytes类型
b = s.encode('utf8')
# print bytes时会将能用ASCII解码的字符直接打印（utf8兼容ASCII）
print(b)
print(len(b))
print(b.decode('utf8'))

4
b'caf\xc3\xa9'
5
café


## 字节概要
Python内置两种基本二进制序列类型：bytes和bytearay，分别为不可变和可变类型，其元素（一个字节）被视为一个uint8（0~255）。

In [10]:
cafe = bytes('café', encoding='utf-8')
print(cafe)
print(cafe[0])
# 一般而言，可切片类型的切片结果仍然是序列本身。元素有自己的类型。像s[0] == s[:1]仅对str成立
print(cafe[1:])
cafe_arr = bytearray(cafe)
print(cafe_arr)
# bytearray可变
cafe_arr[-1] = 1
print(cafe_arr)
# bytes不可变
cafe[-1] = 1

b'caf\xc3\xa9'
99
b'af\xc3\xa9'
bytearray(b'caf\xc3\xa9')
bytearray(b'caf\xc3\x01')


TypeError: 'bytes' object does not support item assignment

对于二进制序列来说，适用于str的endswith、replace、strip、translate、upper等常用方法同样使用与它，此外如果正则re模块使用二进制序列编译，则也能处理二进制序列的识别（一般不会这么做）

其他用于创建bytes或bytearray的方法有：一个str对象和encoding参数、一个提供0~255的可迭代对象、一个实现缓冲协议的对象（bytes、bytearray、memoryview、array.array），此时会复制源对象的字节序列。

In [17]:
import array
# 创建短整型（2字节int）数组
numbers = array.array('h', [-1, 1, 1, 0, 1])
# 表示为10个uint8
octets = bytes(numbers)
print(octets)
print(octets[0])

b'\xff\xff\x01\x00\x01\x00\x00\x00\x01\x00'
255


# 结构体与内存视图
1.struct用于将字节序列bytes、bytearray同时也能处理memoryview将其进行拆解并打包为元组（也可反向生成字节序列）。

2.memoryview使用内存共享机制（可指定展示方式），可以访问其他二进制序列、array.array、数据切片的数据并进行修改，但不会复制字节数据。

In [19]:
import struct
fmt = '<3s3sHH' # 拆为两个3字节序列，两个uint16
with open('OIP.jpg', 'rb') as fp:
    # 创建memoryview不复制数据
    img = memoryview(fp.read())
# 创建memoryview切片（memoryview类型）不复制数据
header = img[:10]
print(bytes(header)) # 创建bytes，需要复制前10个字节的数据
struct.unpack(fmt, header) # 对10bytes的bytes进行结构化拆解并打包为元组

b'\xff\xd8\xff\xe0\x00\x10JFIF'


(b'\xff\xd8\xff', b'\xe0\x00\x10', 17994, 17993)

# 编解码器
python内置很多编解码器

当编码解码器不一致时将会出现解码乱码或编解码异常,多数非utf编码的编码器只能处理unicode中一小部分字符标识，当要编码的文本字符编码器不支持时会抛出UnicodeEncodeError问题。当把二进制序列转为文本时，，遇到无法解码的字节序列会抛出UnicodeDecodeError。

In [22]:
# 不同编码器编码的字节序列
for codec in ['latin1', 'utf8', 'utf16']:
    print(codec, 'El Niño'.encode(codec), sep='\t')

latin1	b'El Ni\xf1o'
utf8	b'El Ni\xc3\xb1o'
utf16	b'\xff\xfeE\x00l\x00 \x00N\x00i\x00\xf1\x00o\x00'


In [25]:
# 使用不同解码器解码相同字节序列
octets = b'Montr\xe9al'
print(octets.decode('cp1252'))
print(octets.decode('iso8859-1'))
print(octets.decode('koi8-r'))
# utf8无法解码，\xe无法解码
print(octets.decode('utf8'))

Montréal
Montréal
MontrИal


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 5: invalid continuation byte

# 文本文件处理最佳实践
## 三明治原则
1.尽早将输入的字节序列（文件）转为字符串（解码）后操作。
2.业务处理只针对字符串
3.尽量晚的转换为字节序列（编码）再进行存储

如：使用open函数读取文件时会做解码，my_file.read()如果不指定以二进制模式读取则会返回解码好的字符串对象，my_file.write(text)中text是要写入的字符串，编码交给write函数。

## 始终指定编解码方式

#### 读取文件和写入文件最好指定统一编解码方法，以免使用默认方式造成不必要的麻烦。